# Customer Churn Analysis
---

## Importing libraries
---

In [ ]:
#!pip install -q duckdb --quiet # installs duckdb
#!pip install skimpy --quiet
import duckdb  # SQL queries on DataFrames
import pandas as pd  # data manipulation and analysis library for dfs
from skimpy import skim # for data profiling

## Adjusting display settings
---

In [ ]:
pd.set_option('display.max_columns', None) # displaying all columns
pd.set_option('display.max_rows', None) # displaying all rows

## Importing data
---

In [ ]:
df_source = pd.read_csv('') # creating a source df with imported data

### Checking df

In [ ]:
df_source.head() # showing 5 rows of data

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### Df basic info

In [ ]:
df_source.info() # checking column data types, null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


## Trasnsforming data
---

In [ ]:
df_source['SeniorCitizen'] = df_source['SeniorCitizen'].map({1: 'Yes', 0: 'No'}) # replacing 0 and 1 for yes and no
df_source['TotalCharges'] = pd.to_numeric(df_source['TotalCharges'], errors='coerce') # converting empty fields to NaN
df_source['gender'] = df_source['gender'].astype('category') # converting column to category type
df_source['SeniorCitizen'] = df_source['SeniorCitizen'].astype('category') # converting column to category type
df_source['Partner'] = df_source['Partner'].astype('category') # converting column to category type
df_source['Dependents'] = df_source['Dependents'].astype('category') # converting column to category type
df_source['PhoneService'] = df_source['PhoneService'].astype('category') # converting column to category type
df_source['MultipleLines'] = df_source['MultipleLines'].astype('category') # converting column to category type
df_source['InternetService'] = df_source['InternetService'].astype('category') # converting column to category type
df_source['OnlineSecurity'] = df_source['OnlineSecurity'].astype('category') # converting column to category type
df_source['OnlineBackup'] = df_source['OnlineBackup'].astype('category') # converting column to category type
df_source['DeviceProtection'] = df_source['DeviceProtection'].astype('category') # converting column to category type
df_source['TechSupport'] = df_source['TechSupport'].astype('category') # converting column to category type
df_source['StreamingTV'] = df_source['StreamingTV'].astype('category') # converting column to category type
df_source['StreamingMovies'] = df_source['StreamingMovies'].astype('category') # converting column to category type
df_source['Contract'] = df_source['Contract'].astype('category') # converting column to category type
df_source['PaperlessBilling'] = df_source['PaperlessBilling'].astype('category') # converting column to category type
df_source['PaymentMethod'] = df_source['PaymentMethod'].astype('category') # converting column to category type
df_source['Churn'] = df_source['Churn'].astype('category') # converting column to category type

In [ ]:
df_source.columns = df_source.columns.str.lower() # Convert all column headers to lowercase

## Checking data summary and basic statistics
---

In [ ]:
skim(df_source)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types               Categories                                        │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓ ┏━━━━━━━━━━━━━━━━━━━━━━━┓                                │
│ ┃ Dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃ ┃ Categorical Variables ┃                                │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩ ┡━━━━━━━━━━━━━━━━━━━━━━━┩                                │
│ │ Number of rows    │ 7043   │ │ category    │ 17    │ │ gender                │                                │
│ │ Number of columns │ 21     │ │ float64     │ 2     │ │ seniorcitizen         │                                │
│ └───────────────────┴────────┘ │ string      │ 1     │ │ partner               │                                │
│                                │ int64       │ 1     │ │ dependents            │                                │
│                                └─────────────┴───────┘ │ phoneservice          │                                │
│                                                        │ multiplelines         │                                │
│                                                        │ internetservice       │                                │
│                                                        │ onlinesecurity        │                                │
│                                                        │ onlinebackup          │                                │
│                                                        │ deviceprotection      │                                │
│                                                        │ techsupport           │                                │
│                                                        │ streamingtv           │                                │
│                                                        │ streamingmovies       │                                │
│                                                        │ contract              │                                │
│                                                        │ paperlessbilling      │                                │
│                                                        │ paymentmethod         │                                │
│                                                        │ churn                 │                                │
│                                                        └───────────────────────┘                                │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━┳━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┓  │
│ ┃ column         ┃ NA ┃ NA %                ┃ mean  ┃ sd    ┃ p0    ┃ p25   ┃ p50   ┃ p75   ┃ p100  ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━╇━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━┩  │
│ │ tenure         │  0 │                   0 │ 32.37 │ 24.56 │     0 │     9 │    29 │    55 │    72 │ █▄▃▃▃▆ │  │
│ │ monthlycharges │  0 │                   0 │ 64.76 │ 30.09 │ 18.25 │  35.5 │ 70.35 │ 89.85 │ 118.8 │ █▃▄▇▇▃ │  │
│ │ totalcharges   │ 11 │ 0.15618344455487718 │  2283 │  2267 │  18.8 │ 401.4 │  1397 │  3795 │  8685 │ █▃▂▂▁▁ │  │
│ └────────────────┴────┴─────────────────────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴────────┘  │
│                                                    category                                                     │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓  │
│ ┃ column                                   ┃ NA       ┃ NA %         ┃ ordered             ┃ unique          ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━

## Dropping unnecessary data
---

In [ ]:
df_source = df_source.dropna(subset=['totalcharges']) # dropping 11 NaN values from TotalCharges

## Analysis
---

### Overall:

#### How many people are churning and what is the % of this group?

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  COUNT(DISTINCT customerID) AS customer_count,
  ROUND(COUNT(DISTINCT customerID) * 100/ (SELECT COUNT(DISTINCT customerID) FROM df_source),2) || '%' AS pct_customer --formatting result to %
FROM
  df_source
GROUP BY
  churn

""").to_df()

df_query

,churn,customer_count,pct_customer
0,Yes,1869,26.58%
1,No,5163,73.42%


### Demographics

#### Gender

In [ ]:
df_query = duckdb.query("""

SELECT
    churn,
    gender AS category,
    COUNT(DISTINCT customerid) AS customer_count,
    ROUND(COUNT(DISTINCT customerid) * 100.0 / Overall_Total, 2) || '%' AS overall_pct, --formatting result to %
    ROUND(COUNT(DISTINCT customerid) * 100.0 / Group_Total, 2) || '%' AS group_pct --formatting result to %
FROM (
    SELECT
        churn,
        gender,
        customerid,
        (SELECT COUNT(DISTINCT customerid) FROM df_source) AS overall_total, -- overall total
        COUNT(DISTINCT customerid) OVER(PARTITION BY gender) AS group_total -- group total
    FROM df_source
) -- subselect to get total values in each group
GROUP BY
  churn,
  gender,
  overall_total,
  group_total
ORDER BY
  gender

""").to_df()

df_query

,churn,category,customer_count,overall_pct,group_pct
0,Yes,Female,939,13.35%,26.96%
1,No,Female,2544,36.18%,73.04%
2,Yes,Male,930,13.23%,26.2%
3,No,Male,2619,37.24%,73.8%


#### Senior citizen

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  seniorcitizen AS category,
  COUNT(DISTINCT customerid) AS customer_count,
  ROUND(COUNT(DISTINCT customerid) * 100.0 / overall_total, 2) || '%' AS overall_pct, --formatting result to %
  ROUND(COUNT(DISTINCT customerid) * 100.0 / group_total, 2) || '%' AS group_pct --formatting result to %
FROM (
  SELECT
    churn,
    seniorcitizen,
    customerid,
    (SELECT COUNT(DISTINCT customerid) FROM df_source) AS overall_total, -- overall total
    COUNT(DISTINCT customerid) OVER(PARTITION BY seniorcitizen) AS group_total -- group total
  FROM
    df_source
) -- subselect to get total values in each group
GROUP BY
  churn,
  seniorcitizen,
  overall_total,
  group_total
ORDER BY
  seniorcitizen

""").to_df()

df_query

,churn,category,customer_count,overall_pct,group_pct
0,Yes,No,1393,19.81%,23.65%
1,No,No,4497,63.95%,76.35%
2,Yes,Yes,476,6.77%,41.68%
3,No,Yes,666,9.47%,58.32%


#### Partner

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  partner AS category,
  COUNT(DISTINCT customerid) AS customer_count,
  ROUND(COUNT(DISTINCT customerid) * 100.0 / overall_total, 2) || '%' AS overall_pct, --formatting result to %
  ROUND(COUNT(DISTINCT customerid) * 100.0 / group_total, 2) || '%' AS group_pct --formatting result to %
FROM (
  SELECT
    churn,
    partner,
    customerid,
    (SELECT COUNT(DISTINCT customerid) FROM df_source) AS overall_total, -- overall total
    COUNT(DISTINCT customerid) OVER(PARTITION BY partner) AS group_total -- group total
  FROM
    df_source
) -- subselect to get total values in each group
GROUP BY
  churn,
  partner,
  overall_total,
  group_total
ORDER BY
  partner

""").to_df()

df_query

,churn,category,customer_count,overall_pct,group_pct
0,Yes,No,1200,17.06%,32.98%
1,No,No,2439,34.68%,67.02%
2,No,Yes,2724,38.74%,80.28%
3,Yes,Yes,669,9.51%,19.72%


#### Dependents

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  dependents AS category,
  COUNT(DISTINCT customerid) AS customer_count,
  ROUND(COUNT(DISTINCT customerid) * 100.0 / overall_total, 2) || '%' AS overall_pct, --formatting result to %
  ROUND(COUNT(DISTINCT customerid) * 100.0 / group_total, 2) || '%' AS group_pct --formatting result to %
FROM (
  SELECT
    churn,
    dependents,
    customerid,
    (SELECT COUNT(DISTINCT customerid) FROM df_source) AS overall_total, -- overall total
    COUNT(DISTINCT customerid) OVER(PARTITION BY dependents) AS group_total -- group total
  FROM
    df_source
) -- subselect to get total values in each group
GROUP BY
  churn,
  dependents,
  overall_total,
  group_total
ORDER BY
  dependents

""").to_df()

df_query

,churn,category,customer_count,overall_pct,group_pct
0,Yes,No,1543,21.94%,31.28%
1,No,No,3390,48.21%,68.72%
2,No,Yes,1773,25.21%,84.47%
3,Yes,Yes,326,4.64%,15.53%


### Services

#### Phone services

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  phoneservice AS category,
  COUNT(DISTINCT customerid) AS customer_count,
  ROUND(COUNT(DISTINCT customerid) * 100.0 / overall_total, 2) || '%' AS overall_pct, --formatting result to %
  ROUND(COUNT(DISTINCT customerid) * 100.0 / group_total, 2) || '%' AS group_pct --formatting result to %
FROM (
  SELECT
    churn,
    phoneservice,
    customerid,
    (SELECT COUNT(DISTINCT customerid) FROM df_source) AS overall_total, -- overall total
    COUNT(DISTINCT customerid) OVER(PARTITION BY phoneservice) AS group_total -- group total
  FROM
    df_source
) -- subselect to get total values in each group
GROUP BY
  churn,
  phoneservice,
  overall_total,
  group_total
ORDER BY
  phoneservice

""").to_df()

df_query

,churn,category,customer_count,overall_pct,group_pct
0,No,No,510,7.25%,75.0%
1,Yes,No,170,2.42%,25.0%
2,No,Yes,4653,66.17%,73.25%
3,Yes,Yes,1699,24.16%,26.75%


##### Checking churned customers with phone services and no tech support

In [ ]:

df_query = duckdb.query("""

SELECT
  COUNT(DISTINCT customerid) AS customers_with_no_support
FROM
  df_source
WHERE
  phoneservice = 'Yes'
  AND churn = 'Yes'
  AND techsupport = 'No'

""").to_df()

df_query

,customers_with_no_support
0,1304


#### Multiple lines

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  multiplelines AS category,
  COUNT(DISTINCT customerid) AS customer_count,
  ROUND(COUNT(DISTINCT customerid) * 100.0 / overall_total, 2) || '%' AS overall_pct, --formatting result to %
  ROUND(COUNT(DISTINCT customerid) * 100.0 / group_total, 2) || '%' AS group_pct --formatting result to %
FROM (
  SELECT
    churn,
    multiplelines,
    customerid,
    (SELECT COUNT(DISTINCT customerid) FROM df_source) AS overall_total, -- overall total
    COUNT(DISTINCT customerid) OVER(PARTITION BY multiplelines) AS group_total -- group total
  FROM
    df_source
) -- subselect to get total values in each group
GROUP BY
  churn,
  multiplelines,
  overall_total,
  group_total
ORDER BY
  multiplelines

""").to_df()

df_query

,churn,category,customer_count,overall_pct,group_pct
0,No,No,2536,36.06%,74.92%
1,Yes,No,849,12.07%,25.08%
2,Yes,No phone service,170,2.42%,25.0%
3,No,No phone service,510,7.25%,75.0%
4,Yes,Yes,850,12.09%,28.65%
5,No,Yes,2117,30.11%,71.35%


##### Checking churned customers in multiple lines category and no tech support

In [ ]:
df_query = duckdb.query("""

SELECT
  COUNT(DISTINCT customerid) AS customers_with_no_support
FROM
  df_source
WHERE
  multiplelines IN ('No','Yes')
  AND churn = 'Yes'
  AND techsupport = 'No'

""").to_df()

df_query

,customers_with_no_support
0,1304


#### Internet Service

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  internetservice AS category,
  COUNT(DISTINCT customerid) AS customer_count,
  ROUND(COUNT(DISTINCT customerid) * 100.0 / overall_total, 2) || '%' AS overall_pct, --formatting result to %
  ROUND(COUNT(DISTINCT customerid) * 100.0 / group_total, 2) || '%' AS group_pct --formatting result to %
FROM (
  SELECT
    churn,
    internetservice,
    customerid,
    (SELECT COUNT(DISTINCT customerid) FROM df_source) AS overall_total, -- overall total
    COUNT(DISTINCT customerid) OVER(PARTITION BY internetservice) AS group_total -- group total
  FROM
    df_source
) -- subselect to get total values in each group
GROUP BY
  churn,
  internetservice,
  overall_total,
  group_total
ORDER BY
  internetservice

""").to_df()

df_query

,churn,category,customer_count,overall_pct,group_pct
0,No,DSL,1957,27.83%,81.0%
1,Yes,DSL,459,6.53%,19.0%
2,Yes,Fiber optic,1297,18.44%,41.89%
3,No,Fiber optic,1799,25.58%,58.11%
4,No,No,1407,20.01%,92.57%
5,Yes,No,113,1.61%,7.43%


##### Checking churned customers with Fiber Optic internet and no tech support

In [ ]:
df_query = duckdb.query("""

SELECT
  COUNT(DISTINCT customerid) AS customers_with_no_support
FROM
  df_source
WHERE
  internetservice = 'Fiber optic'
  AND churn = 'Yes'
  AND techsupport = 'No'

""").to_df()

df_query

,customers_with_no_support
0,1101


#### Online Security

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  onlinesecurity AS category,
  COUNT(DISTINCT customerid) AS customer_count,
  ROUND(COUNT(DISTINCT customerid) * 100.0 / overall_total, 2) || '%' AS overall_pct, --formatting result to %
  ROUND(COUNT(DISTINCT customerid) * 100.0 / group_total, 2) || '%' AS group_pct --formatting result to %
FROM (
  SELECT
    churn,
    onlinesecurity,
    customerid,
    (SELECT COUNT(DISTINCT customerid) FROM df_source) AS overall_total, -- overall total
    COUNT(DISTINCT customerid) OVER(PARTITION BY onlinesecurity) AS group_total -- group total
  FROM
    df_source
) -- subselect to get total values in each group
GROUP BY
  churn,
  onlinesecurity,
  overall_total,
  group_total
ORDER BY
  onlinesecurity

""").to_df()

df_query

,churn,category,customer_count,overall_pct,group_pct
0,Yes,No,1461,20.78%,41.78%
1,No,No,2036,28.95%,58.22%
2,No,No internet service,1407,20.01%,92.57%
3,Yes,No internet service,113,1.61%,7.43%
4,Yes,Yes,295,4.2%,14.64%
5,No,Yes,1720,24.46%,85.36%


#### Online Backup

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  onlinebackup AS category,
  COUNT(DISTINCT customerid) AS customer_count,
  ROUND(COUNT(DISTINCT customerid) * 100.0 / overall_total, 2) || '%' AS overall_pct, --formatting result to %
  ROUND(COUNT(DISTINCT customerid) * 100.0 / group_total, 2) || '%' AS group_pct --formatting result to %
FROM (
  SELECT
    churn,
    onlinebackup,
    customerid,
    (SELECT COUNT(DISTINCT customerid) FROM df_source) AS overall_total, -- overall total
    COUNT(DISTINCT customerid) OVER(PARTITION BY onlinebackup) AS group_total -- group total
  FROM
    df_source
) -- subselect to get total values in each group
GROUP BY
  churn,
  onlinebackup,
  overall_total,
  group_total
ORDER BY
  onlinebackup

""").to_df()

df_query

,churn,category,customer_count,overall_pct,group_pct
0,Yes,No,1233,17.53%,39.94%
1,No,No,1854,26.37%,60.06%
2,No,No internet service,1407,20.01%,92.57%
3,Yes,No internet service,113,1.61%,7.43%
4,Yes,Yes,523,7.44%,21.57%
5,No,Yes,1902,27.05%,78.43%


#### Device Protection

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  deviceprotection AS category,
  COUNT(DISTINCT customerid) AS customer_count,
  ROUND(COUNT(DISTINCT customerid) * 100.0 / overall_total, 2) || '%' AS overall_pct, -- formatting result to %
  ROUND(COUNT(DISTINCT customerid) * 100.0 / group_total, 2) || '%' AS group_pct -- formatting result to %
FROM (
  SELECT
    churn,
    deviceprotection,
    customerid,
    (SELECT COUNT(DISTINCT customerid) FROM df_source) AS overall_total, -- overall total
    COUNT(DISTINCT customerid) OVER(PARTITION BY deviceprotection) AS group_total -- group total
  FROM
    df_source
) -- subselect to get total values in each group
GROUP BY
  churn,
  deviceprotection,
  overall_total,
  group_total
ORDER BY
  deviceprotection

""").to_df()

df_query

,churn,category,customer_count,overall_pct,group_pct
0,No,No,1883,26.78%,60.86%
1,Yes,No,1211,17.22%,39.14%
2,Yes,No internet service,113,1.61%,7.43%
3,No,No internet service,1407,20.01%,92.57%
4,No,Yes,1873,26.64%,77.46%
5,Yes,Yes,545,7.75%,22.54%


#### Tech Support

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  techsupport AS category,
  COUNT(DISTINCT customerid) AS customer_count,
  ROUND(COUNT(DISTINCT customerid) * 100.0 / overall_total, 2) || '%' AS overall_pct, -- formatting result to %
  ROUND(COUNT(DISTINCT customerid) * 100.0 / group_total, 2) || '%' AS group_pct -- formatting result to %
FROM (
  SELECT
    churn,
    techsupport,
    customerid,
    (SELECT COUNT(DISTINCT customerid) FROM df_source) AS overall_total, -- overall total
    COUNT(DISTINCT customerid) OVER(PARTITION BY techsupport) AS group_total -- group total
  FROM
    df_source
) -- subselect to get total values in each group
GROUP BY
  churn,
  techsupport,
  overall_total,
  group_total
ORDER BY
  techsupport

""").to_df()

df_query

,churn,category,customer_count,overall_pct,group_pct
0,Yes,No,1446,20.56%,41.65%
1,No,No,2026,28.81%,58.35%
2,No,No internet service,1407,20.01%,92.57%
3,Yes,No internet service,113,1.61%,7.43%
4,No,Yes,1730,24.6%,84.8%
5,Yes,Yes,310,4.41%,15.2%


#### Streaming TV

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  streamingtv AS category,
  COUNT(DISTINCT customerid) AS customer_count,
  ROUND(COUNT(DISTINCT customerid) * 100.0 / overall_total, 2) || '%' AS overall_pct, -- formatting result to %
  ROUND(COUNT(DISTINCT customerid) * 100.0 / group_total, 2) || '%' AS group_pct -- formatting result to %
FROM (
  SELECT
    churn,
    streamingtv,
    customerid,
    (SELECT COUNT(DISTINCT customerid) FROM df_source) AS overall_total, -- overall total
    COUNT(DISTINCT customerid) OVER(PARTITION BY streamingtv) AS group_total -- group total
  FROM
    df_source
) -- subselect to get total values in each group
GROUP BY
  churn,
  streamingtv,
  overall_total,
  group_total
ORDER BY
  streamingtv

""").to_df()

df_query

,churn,category,customer_count,overall_pct,group_pct
0,No,No,1867,26.55%,66.46%
1,Yes,No,942,13.4%,33.54%
2,Yes,No internet service,113,1.61%,7.43%
3,No,No internet service,1407,20.01%,92.57%
4,Yes,Yes,814,11.58%,30.11%
5,No,Yes,1889,26.86%,69.89%


#### Streaming Movies

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  streamingmovies AS category,
  COUNT(DISTINCT customerid) AS customer_count,
  ROUND(COUNT(DISTINCT customerid) * 100.0 / overall_total, 2) || '%' AS overall_pct, -- formatting result to %
  ROUND(COUNT(DISTINCT customerid) * 100.0 / group_total, 2) || '%' AS group_pct -- formatting result to %
FROM (
  SELECT
    churn,
    streamingmovies,
    customerid,
    (SELECT COUNT(DISTINCT customerid) FROM df_source) AS overall_total, -- overall total
    COUNT(DISTINCT customerid) OVER(PARTITION BY streamingmovies) AS group_total -- group total
  FROM
    df_source
) -- subselect to get total values in each group
GROUP BY
  churn,
  streamingmovies,
  overall_total,
  group_total
ORDER BY
  streamingmovies

""").to_df()

df_query

,churn,category,customer_count,overall_pct,group_pct
0,No,No,1843,26.21%,66.27%
1,Yes,No,938,13.34%,33.73%
2,Yes,No internet service,113,1.61%,7.43%
3,No,No internet service,1407,20.01%,92.57%
4,Yes,Yes,818,11.63%,29.95%
5,No,Yes,1913,27.2%,70.05%


### Financial

#### Contract

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  contract AS category,
  COUNT(DISTINCT customerid) AS customer_count,
  ROUND(COUNT(DISTINCT customerid) * 100.0 / overall_total, 2) || '%' AS overall_pct, -- formatting result to %
  ROUND(COUNT(DISTINCT customerid) * 100.0 / group_total, 2) || '%' AS group_pct -- formatting result to %
FROM (
  SELECT
    churn,
    contract,
    customerid,
    (SELECT COUNT(DISTINCT customerid) FROM df_source) AS overall_total, -- overall total
    COUNT(DISTINCT customerid) OVER(PARTITION BY contract) AS group_total -- group total
  FROM
    df_source
) -- subselect to get total values in each group
GROUP BY
  churn,
  contract,
  overall_total,
  group_total
ORDER BY
  contract

""").to_df()

df_query

,churn,category,customer_count,overall_pct,group_pct
0,Yes,Month-to-month,1655,23.54%,42.71%
1,No,Month-to-month,2220,31.57%,57.29%
2,Yes,One year,166,2.36%,11.28%
3,No,One year,1306,18.57%,88.72%
4,No,Two year,1637,23.28%,97.15%
5,Yes,Two year,48,0.68%,2.85%


#### Tenure

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  quantile(tenure, 0.25) AS tenure_q1,
  quantile(tenure, 0.50) AS tenure_q2,
  quantile(tenure, 0.75) AS tenure_q3
FROM
  df_source
GROUP BY
  churn

""").to_df()

df_query

,churn,tenure_q1,tenure_q2,tenure_q3
0,No,15,38,61
1,Yes,2,10,29


#### Paperless Billing

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  paperlessbilling AS category,
  COUNT(DISTINCT customerid) AS customer_count,
  ROUND(COUNT(DISTINCT customerid) * 100.0 / overall_total, 2) || '%' AS overall_pct, -- formatting result to %
  ROUND(COUNT(DISTINCT customerid) * 100.0 / group_total, 2) || '%' AS group_pct -- formatting result to %
FROM (
  SELECT
    churn,
    paperlessbilling,
    customerid,
    (SELECT COUNT(DISTINCT customerid) FROM df_source) AS overall_total, -- overall total
    COUNT(DISTINCT customerid) OVER(PARTITION BY paperlessbilling) AS group_total -- group total
  FROM
    df_source
) -- subselect to get total values in each group
GROUP BY
  churn,
  paperlessbilling,
  overall_total,
  group_total
ORDER BY
  paperlessbilling

""").to_df()

df_query

,churn,category,customer_count,overall_pct,group_pct
0,No,No,2395,34.06%,83.62%
1,Yes,No,469,6.67%,16.38%
2,Yes,Yes,1400,19.91%,33.59%
3,No,Yes,2768,39.36%,66.41%


In [ ]:
df_query = duckdb.query("""

SELECT
  COUNT(DISTINCT customerid) AS monthly_contract_customer_count
FROM
  df_source
WHERE
  paperlessbilling = 'Yes'
  AND churn = 'Yes'
  AND contract = 'Month-to-month'

""").to_df()

df_query

,monthly_contract_customer_count
0,1249


#### Payment method

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  paymentmethod AS category,
  COUNT(DISTINCT customerid) AS customer_count,
  ROUND(COUNT(DISTINCT customerid) * 100.0 / overall_total, 2) || '%' AS overall_pct, -- formatting result to %
  ROUND(COUNT(DISTINCT customerid) * 100.0 / group_total, 2) || '%' AS group_pct -- formatting result to %
FROM (
  SELECT
    churn,
    paymentmethod,
    customerid,
    (SELECT COUNT(DISTINCT customerid) FROM df_source) AS overall_total, -- overall total
    COUNT(DISTINCT customerid) OVER(PARTITION BY paymentmethod) AS group_total -- group total
  FROM
    df_source
) -- subselect to get total values in each group
GROUP BY
  churn,
  paymentmethod,
  overall_total,
  group_total
ORDER BY
  paymentmethod

""").to_df()

df_query

,churn,category,customer_count,overall_pct,group_pct
0,No,Bank transfer (automatic),1284,18.26%,83.27%
1,Yes,Bank transfer (automatic),258,3.67%,16.73%
2,No,Credit card (automatic),1289,18.33%,84.75%
3,Yes,Credit card (automatic),232,3.3%,15.25%
4,Yes,Electronic check,1071,15.23%,45.29%
5,No,Electronic check,1294,18.4%,54.71%
6,Yes,Mailed check,308,4.38%,19.2%
7,No,Mailed check,1296,18.43%,80.8%


#### Monthly charges

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  quantile(monthlycharges, 0.25) AS monthly_charges_q1,
  quantile(monthlycharges, 0.50) AS monthly_charges_q2,
  quantile(monthlycharges, 0.75) AS monthly_charges_q3
FROM
  df_source
GROUP BY
  churn

""").to_df()

df_query

,churn,monthly_charges_q1,monthly_charges_q2,monthly_charges_q3
0,No,25.10,64.45,88.5
1,Yes,56.15,79.65,94.2


#### Total charges

In [ ]:
df_query = duckdb.query("""

SELECT
  churn,
  quantile(totalcharges, 0.25) AS total_charges_q1,
  quantile(totalcharges, 0.50) AS total_charges_q2,
  quantile(totalcharges, 0.75) AS total_charges_q3
FROM
  df_source
GROUP BY
  churn

""").to_df()

df_query

,churn,total_charges_q1,total_charges_q2,total_charges_q3
0,No,577.15,1683.60,4264.25
1,Yes,134.50,703.55,2331.30


### Information value

In [ ]:
df_query_iv = duckdb.query("""

WITH total_counts AS (
    SELECT
        SUM(CASE WHEN churn = 'No' THEN 1 ELSE 0 END) AS total_no_churn,
        SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS total_churn
    FROM df_source
),
-- all categorical variables
stacked AS (
    SELECT 'Gender' AS variable, gender AS y_n, churn
    FROM df_source
    UNION ALL
    SELECT 'Senior Citizen', seniorcitizen, churn FROM df_source
    UNION ALL
    SELECT 'Partner', partner, churn FROM df_source
    UNION ALL
    SELECT 'Dependents', dependents, churn FROM df_source
    UNION ALL
    SELECT 'Phone Service', phoneservice, churn FROM df_source
    UNION ALL
    SELECT 'Multiple Lines', multiplelines, churn FROM df_source
    UNION ALL
    SELECT 'Internet Service', internetservice, churn FROM df_source
    UNION ALL
    SELECT 'Online Security', onlinesecurity, churn FROM df_source
    UNION ALL
    SELECT 'Online Backup', onlinebackup, churn FROM df_source
    UNION ALL
    SELECT 'Device Protection', deviceprotection, churn FROM df_source
    UNION ALL
    SELECT 'Tech Support', techsupport, churn FROM df_source
    UNION ALL
    SELECT 'Streaming TV', streamingtv, churn FROM df_source
    UNION ALL
    SELECT 'Streaming Movies', streamingmovies, churn FROM df_source
    UNION ALL
    SELECT 'Contract', contract, churn FROM df_source
    UNION ALL
    SELECT 'Paperless Billing', paperlessbilling, churn FROM df_source
    UNION ALL
    SELECT 'Payment Method', paymentmethod, churn FROM df_source
),
-- aggregating data
agg AS (
    SELECT
        variable,
        y_n,
        SUM(CASE WHEN churn = 'No' THEN 1 ELSE 0 END) AS no_churn,
        SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS churn
    FROM
      stacked
    GROUP BY
      variable, y_n
),
-- calculating WOE e IV
iv_calc AS (
    SELECT
        a.variable,
        a.y_n,
        no_churn,
        churn,
        no_churn * 1.0 / t.total_no_churn AS dist_no_churn,
        churn * 1.0 / t.total_churn AS dist_churn,
        LN(NULLIF(no_churn / t.total_no_churn, 0) /
           NULLIF(churn / t.total_churn, 0)) AS woe,
        (no_churn / t.total_no_churn - churn / t.total_churn) *
           LN(NULLIF(no_churn / t.total_no_churn, 0) /
              NULLIF(churn / t.total_churn, 0)) AS iv_part
    FROM
      agg a
    CROSS JOIN
      total_counts t
)
SELECT
    variable,
    y_n,
    no_churn,
    churn,
    dist_no_churn,
    dist_churn,
    woe,
    iv_part,
    ROUND(SUM(iv_part) OVER (PARTITION BY variable), 3) AS information_value -- sum values to get iv
FROM
  iv_calc
ORDER BY
  information_value DESC

""").to_df()

df_query_iv

,variable,y_n,no_churn,churn,dist_no_churn,dist_churn,woe,iv_part,information_value
0,Contract,Two year,1637.0,48.0,0.317064,0.025682,2.513305,0.732331,1.233
1,Contract,Month-to-month,2220.0,1655.0,0.429983,0.885500,-0.722408,0.329070,1.233
2,Contract,One year,1306.0,166.0,0.252954,0.088818,1.046622,0.171789,1.233
3,Online Security,Yes,1720.0,295.0,0.333140,0.157838,0.746990,0.130948,0.715
4,Online Security,No internet service,1407.0,113.0,0.272516,0.060460,1.505713,0.319295,0.715
5,Online Security,No,2036.0,1461.0,0.394344,0.781701,-0.684248,0.265048,0.715
6,Tech Support,No,2026.0,1446.0,0.392408,0.773676,-0.678852,0.258825,0.697
7,Tech Support,Yes,1730.0,310.0,0.335077,0.165864,0.703190,0.118988,0.697
8,Tech Support,No internet service,1407.0,113.0,0.272516,0.060460,1.505713,0.319295,0.697
9,Internet Service,DSL,1957.0,459.0,0.379043,0.245586,0.434003,0.057921,0.615


#### Summarizing values

In [ ]:
df_query = duckdb.query("""

SELECT DISTINCT
  variable,
  information_value
FROM
  df_query_iv
ORDER BY
  information_value DESC

""").to_df()

df_query

,variable,information_value
0,Contract,1.233
1,Online Security,0.715
2,Tech Support,0.697
3,Internet Service,0.615
4,Online Backup,0.526
5,Device Protection,0.498
6,Payment Method,0.456
7,Streaming Movies,0.380
8,Streaming TV,0.379
9,Paperless Billing,0.202


#### Checking lowest woe values

In [ ]:
df_query = duckdb.query("""

SELECT DISTINCT
  variable,
  y_n,
  woe,
  information_value
FROM
  df_query_iv
ORDER BY
  woe ASC

""").to_df()

df_query

,variable,y_n,woe,information_value
0,Payment Method,Electronic check,-0.826969,0.456
1,Contract,Month-to-month,-0.722408,1.233
2,Internet Service,Fiber optic,-0.688937,0.615
3,Online Security,No,-0.684248,0.715
4,Senior Citizen,Yes,-0.680242,0.105
5,Tech Support,No,-0.678852,0.697
6,Online Backup,No,-0.608219,0.526
7,Device Protection,No,-0.574694,0.498
8,Streaming Movies,No,-0.340714,0.380
9,Paperless Billing,Yes,-0.334461,0.202
